# From REINFORCE to PPO:

---

In this notebook, we will train REINFORCE with OpenAI Gym's Cartpole environment and cover the concepts behind REINFORCE. Then we will discuss its issues and finally learn how we can adress them using Proximal Policy Optimization (PPO).

### 1. Import the Necessary Packages

In [6]:
import gym
gym.logger.set_level(40) # suppress warnings (please remove if gives error)
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

### 2. Define the Architecture of the Policy

In [7]:
env = gym.make('CartPole-v0')
env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

# Set the device on which a torch.Tensor will be allocated.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define your NN architecture
class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        # Set the corresponding action space size in the output layer
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Re-scale the raw scores (aka logits) so that the elements lie in range [0..1] 
        return F.softmax(x, dim=1)
    
    def act(self, state):
        # Convert state from numpy array to torch tensor and assign to it the device where it will be allocated
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        # Given a state, return the action probabilities from and .cpu() to copy its values in CPU memory
        probs = self.forward(state).cpu()
        # Categorical creates a categoricaldistribution parametrized by either probs or logits
        # A categorical distribution is a discrete probability distribution that describes the possible results 
        # of a random variable that can take on one of K possible categories, with the probability of each category
        # separately specified
        # So here the random variable can take one of the ActionSpaceSize possible actions, and the given probabilities 
        # are the output of the NN
        m = Categorical(probs)
        # Given the probability distribution m the sample method will sample an action
        action = m.sample()
        # Log_prob method will calculate the log_probabilty of the sample action under the distribution m
        # item() returns the value of a tensor as a standard python number
        return action.item(), m.log_prob(action)

observation space: Box(4,)
action space: Discrete(2)


### 3. Train the Agent with REINFORCE

In [ ]:
# Redirect the allocation of NN tensors 
policy = Policy().to(device)
# Set the optimizer
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def reinforce(n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Set a maximum timelimit for episode
        for t in range(max_t):
            # Given the state and under the policy theta return the sample action and its log prob
            action, log_prob = policy.act(state)
            # Need to collect all log probs for further calculation
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            # as well as rewards
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        # According to the range of rewards prepare gammas' values
        discounts = [gamma**i for i in range(len(rewards)+1)]
        # Total discounted reward R(𝜏) = Σi=0 γi ri
        R = sum([a*b for a,b in zip(discounts, rewards)])
        
        policy_loss = []
        for log_prob in saved_log_probs:
            # Loss equal to the negated log-probability of the action taken multiplied by the total discounted
            # reward of tau Loss 
            policy_loss.append(-log_prob * R)
        # Sum the losses 
        policy_loss = torch.cat(policy_loss).sum()
        
        optimizer.zero_grad()
        # Calculate the gradient
        policy_loss.backward()
        # Update the gradient
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores
    
scores = reinforce()

In [5]:
from IPython.display import Latex, Image

### 1. Policy Gradient is Noisy, gradients have High variance...
* We take one trajectory, calculate the Noise, compute the gradient, update the policy and so on ...
Recall that$$\nabla_{\theta}U(\theta)= E_{\tau \sim \pi_{\theta}(\tau) } \nabla_{\theta} \log \pi_{\theta}(\tau) 
r(\tau)
$$
Here, tau is the trajectory (the sequences of states st and actions at) the agent takes through a given sample, say from timestep 1 to T
$$  \log \pi_{\theta}(\tau) = \sum_{t=1}^{t=T} \log \pi_{\theta}(a_{t} | s_{t})$$  and $$r(\tau) = \sum_{t=1}^{t=T} r(s_{t},a_{t})$$
The typical way we compute this gradient is to sample a bunch of trajectories from the current policy 𝜏 ∼ 𝜋𝜃(𝜏) and average their values (the value we’re taking the expectation over). Then the computed gradient becomes dependent on the randomly sampled trajectories: $$\nabla_{\theta}U(\theta) \approx \frac{1}{N} \sum_{i=1}^{N} \nabla_{\theta} \log \pi_{\theta}(\tau)  r(\tau)$$
As a result, it’s going to have variance, since its values depend on the sampled trajectories.     


![Graph from CS294](img/SampledTrajectories.png)  

This variance can be quite high. This is because each trajectory can take very different paths depending on the states visited and actions sampled from your policy, as shown in the graph above. This means that ∇𝜃 log 𝜋𝜃(𝜏) and  𝑟(𝜏) take on wildly different values depending on the trajectory.      

Of course, the first easiest option is that if we scale N→∞, the average values will approach your true expected value, but we can’t afford to make N too high due to computational cost.

### 2. Credit Assignment:
* If the current policy gradient can be written as:

$$\nabla_{\theta}U(\theta) \approx \frac{1}{N} \sum_{i=0}^{N}   \sum_{t=1}^{t=T} \log \pi_{\theta}(a_{t} | s_{t})\sum_{t=1}^{t=T} r(s_{t},a_{t})$$

Then we can instead write it as:

$$\nabla_{\theta}U(\theta) \approx \frac{1}{N} \sum_{i=0}^{N}   \sum_{t=1}^{t=T} \log \pi_{\theta}(a_{t} | s_{t})\sum_{t=t^{'}}^{t=T} r(s_{t^{'}},a_{t^{'}})$$
We switched from t to t. As we are in Markov process, this basically says that the action at time-step t can only affect the future reward, so the past reward shouldn’t be contributing to the policy gradient. So a better policy gradient would simply have the future reward as the coefficient.  
This allows:
* To properly assign credits to the actions at.
* Also reduces the number of random variables and consequently reduces the variance of the gradients.

### 3. The update process is inefficient:
1. Run the policy once.
2. Update once.
3. Throw away the trajectory.  

What about recycling the trajectories and modifying them so that they become representative of the new policy and then use the recycled trajectories to compute the gradients and update our policy again and again.  

To mitigate this inefficiency issue, we should use Importing Sampling.
It allows us to estimate properties of a particular distribution while only having samples generated from a different distribution than the distribution of interest.  

In our case, the generated trajectories with the policy 𝜋𝜃 have a probability P(𝜏,𝜃) to be sampled. If by chance the same trajectory can be sampled under another updated policy with a different probability P(𝜏,𝜃'). If we want to compute the average of some quantity, say f(𝜏), mathematically it's equivalent to:  

$$ \sum_{\tau} P(\tau;\theta^{'}) f(\tau) $$
We could modify this equation by multiplying and dividing by the same number which is 𝑃(𝜏;𝜃′) and rearrange the terms:  
$$ \sum_{\tau} P(\tau;\theta) \frac{P(\tau;\theta^{'})}{ P(\tau;\theta)}f(\tau) $$
This allows to interpret the equation as the coefficient for sampling under the old policy, with an extra re-weighting factor in addition to just averaging.  
!! We need to make sure that the re-weighting factor is not too far from 1.

### 4. The surrogate function:
With all previous ideas taken into consideration, if we want to update our policy 𝜋𝜃′ and we only have trajectories generated by an older policy we use importance sampling and compute the gradient as follows:
$$ g = \frac{ P(\tau;\theta^{'})}{ P(\tau;\theta)} \sum_{t} \frac{\nabla_{\theta^{'}} \pi_{\theta^{'}}(a_{t}|s_{t})}{ \pi_{\theta^{'}}(a_{t}|s_{t})} R_{t}^{future}$$  
The re-weighting factor is just the product of all the policy accross each step:
$$ g = \sum_{t}  \frac{... \pi_{\theta^{'}}(a_{t}|s_{t}) ...} {... \pi_{\theta}(a_{t}|s_{t}) ...}  \frac{\nabla_{\theta^{'}} \pi_{\theta^{'}}(a_{t}|s_{t})}{ \pi_{\theta^{'}}(a_{t}|s_{t})} R_{t}^{future}$$  
And if we rearrange the equation a little bit more: 
$$ g = \sum_{t}  \frac{... \pi_{\theta^{'}}(a_{t}|s_{t}) ...} {... \pi_{\theta^{'}}(a_{t}|s_{t})  ...}  \frac{\nabla_{\theta^{'}} \pi_{\theta}(a_{t}|s_{t})}{ \pi_{\theta}(a_{t}|s_{t})} R_{t}^{future}$$ 
We can cancel the terms on the left, but still we are left with a product of policies at different times. And here where PPO comes in, if the old and the current policy are close to each other, all these . . . factors will be pretty close to one, then we can ignore them and the equation simplifies even further:
$$ g = \sum_{t} \frac{\nabla_{\theta^{'}} \pi_{\theta}(a_{t}|s_{t})}{ \pi_{\theta}(a_{t}|s_{t})} R_{t}^{future}$$ 
Now that we have an approximate form of the gradient, we can think of it as a gradient of a new object which is: 
$$ g =\nabla_{\theta^{'}} \sum_{t} \frac{ \pi_{\theta^{'}}(a_{t}|s_{t})}{ \pi_{\theta}(a_{t}|s_{t})} R_{t}^{future}$$
$$ g =\nabla_{\theta^{'}} L_{sur}(\theta^{'},\theta)$$  
**To sum up here:** The vanilla policy gradient method uses the log probability of the actions log 𝜋𝜃(𝑎𝑡|𝑠𝑡) to trace the impact of the actions, but here another function is doing this. 𝐿𝑠𝑢𝑟(𝜃′,𝜃) uses the probability of the action under the current policy 𝜋𝜃′(𝑎|𝑠), divided by the probability of the action under your previous policy 𝜋𝜃(𝑎|𝑠).  

### 5. The clipped surrogate function:
if we keep re-using old trajectories and updating our policy, at some point the new policy might become different enough from the old one, that may result in having all the approximations we made could become invalid. Also, the raio  will tend to be really big and lead to taking big gradient steps that might wreck the policy after the update.
Instead of adding KL-divergence such as in TRPO, what about building these properties into the objective function - itself:
$$L_{sur}^{clip}(\theta,\theta^{'}) = \sum_{t} min(  \frac{ \pi_{\theta^{'}}(a_{t}|s_{t})}{ \pi_{\theta}(a_{t}|s_{t})} R_{t}^{future}, clip_{\epsilon}(\frac{ \pi_{\theta^{'}}(a_{t}|s_{t})}{ \pi_{\theta}(a_{t}|s_{t})}) R_{t}^{future}   )$$  
- The first term inside the minimization is the same ratio we saw in the previous 𝐿𝑠𝑢𝑟(𝜃′,𝜃) objective (it's the same as TRPO objective).
- The second term is a version where the the ratio is clipped between (1 - 𝜖, 1 + 𝜖). (in the paper they state a good value for 𝜖 is about 0.2, so r can vary between ~(0.8, 1.2)). 
- Then, finally, the minimization of both of these terms is taken.

### 6. PPO algorithm:
![PPO](img/PPO.png)

1. Collect some trajectories based on some policy 𝜋𝜃(𝑎|𝑠), and initialize θ′=θ
2. Next, compute the gradient of the clipped surrogate function using the trajectories
3. Update θ′ using gradient ascent θ′← θ′+α∇θ′Lsurclip(θ′,θ)
4. Then we repeat step 2-3 without generating new trajectories. Typically, step 2-3 are only repeated a few times
5. Set θ=θ′, go back to step 1, repeat.
